In [1]:
# ============================================================
# FULL SCRIPT (FIXED): Auto-generate missing harmonized CSVs
# + Swin Transformer Tiny (swin_t) backbone
#
# This fixes your error:
# FileNotFoundError: /kaggle/working/harmonized_labels/ODIR_train.csv
# by generating /kaggle/working/harmonized_labels/*.csv automatically
# if they don't exist.
# ============================================================

import os
import random
from typing import Dict

import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, ConcatDataset

import torchvision.transforms as transforms
import torchvision.models as models

from sklearn.metrics import roc_auc_score, f1_score, roc_curve, average_precision_score
import matplotlib.pyplot as plt
from tqdm import tqdm


# ============================================================
# 1) HARMONIZATION
# ============================================================

def harmonize_odir(split: str) -> pd.DataFrame:
    base_path = "/kaggle/input/odir-clr/ODIR_CLR"

    if split == "train":
        img_dir = f"{base_path}/Training_ Set/train_images"
        label_file = f"{base_path}/Training_ Set/train_annotation.xlsx"
    elif split == "val":
        img_dir = f"{base_path}/Validation_set/val_images"
        label_file = f"{base_path}/Validation_set/val_annotation.xlsx"
    else:
        img_dir = f"{base_path}/Test_Set/test_images"
        label_file = f"{base_path}/Test_Set/test_annotation.xlsx"

    df = pd.read_excel(label_file)
    label_cols = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']
    harmonized_rows = []

    for _, row in df.iterrows():
        patient_id = row['ID']
        labels = {col: int(row[col]) for col in label_cols}

        if pd.notna(row.get('Left-Fundus', np.nan)) and str(row['Left-Fundus']).strip():
            left_path = os.path.join(img_dir, f"{patient_id}_left.jpg")
            harmonized_rows.append({
                'image_path': left_path,
                'dataset': 'ODIR',
                'split': split,
                'ID': patient_id,
                **labels
            })

        if pd.notna(row.get('Right-Fundus', np.nan)) and str(row['Right-Fundus']).strip():
            right_path = os.path.join(img_dir, f"{patient_id}_right.jpg")
            harmonized_rows.append({
                'image_path': right_path,
                'dataset': 'ODIR',
                'split': split,
                'ID': patient_id,
                **labels
            })

    return pd.DataFrame(harmonized_rows)


def harmonize_rfmid_v1(split: str) -> pd.DataFrame:
    base_path = "/kaggle/input/retinal-disease-classification"

    if split == "train":
        img_dir = f"{base_path}/Training_Set/Training_Set/Training"
        label_file = f"{base_path}/Training_Set/Training_Set/RFMiD_Training_Labels.csv"
    elif split == "val":
        img_dir = f"{base_path}/Evaluation_Set/Evaluation_Set/Validation"
        label_file = f"{base_path}/Evaluation_Set/Evaluation_Set/RFMiD_Validation_Labels.csv"
    else:
        img_dir = f"{base_path}/Test_Set/Test_Set/Test"
        label_file = f"{base_path}/Test_Set/Test_Set/RFMiD_Testing_Labels.csv"

    df = pd.read_csv(label_file)

    all_disease_cols = ['DR', 'ARMD', 'MH', 'DN', 'MYA', 'BRVO', 'TSLN',
                        'ERM', 'LS', 'MS', 'CSR', 'ODC', 'CRVO', 'TV', 'AH',
                        'ODP', 'ODE', 'ST', 'AION', 'PT', 'RT', 'RS', 'CRS',
                        'EDN', 'RPEC', 'MHL', 'RP', 'CWS', 'CB', 'ODPM',
                        'PRH', 'MNF', 'HR', 'CRAO', 'TD', 'CME', 'PTCR', 'CF',
                        'VH', 'MCA', 'VS', 'BRAO', 'PLQ', 'HPED', 'CL']

    harmonized_rows = []

    for _, row in df.iterrows():
        image_id = row['ID']

        # Try common extensions
        image_path = None
        for ext in ['.png', '.jpg', '.jpeg', '.JPG', '.PNG', '.JPEG']:
            candidate = os.path.join(img_dir, f"{image_id}{ext}")
            if os.path.exists(candidate):
                image_path = candidate
                break
        if image_path is None:
            # last fallback (some datasets store exact filename)
            image_path = os.path.join(img_dir, str(image_id))

        # Mapping
        N = 1 if row['Disease_Risk'] == 0 else 0
        D = 1 if row.get('DR', 0) == 1 else 0
        G = 1 if row.get('ODC', 0) == 1 else 0
        C = 1 if row.get('MH', 0) == 1 else 0
        A = 1 if row.get('ARMD', 0) == 1 else 0
        H = 1 if row.get('HR', 0) == 1 else 0
        M = 1 if row.get('MYA', 0) == 1 else 0

        used_for_mapping = ['DR', 'ODC', 'MH', 'ARMD', 'HR', 'MYA']
        other_cols = [col for col in all_disease_cols if col not in used_for_mapping]
        O = 1 if any(row.get(col, 0) == 1 for col in other_cols) else 0

        harmonized_rows.append({
            'image_path': image_path,
            'dataset': 'RFMiD_v1',
            'split': split,
            'ID': image_id,
            'N': N, 'D': D, 'G': G, 'C': C, 'A': A, 'H': H, 'M': M, 'O': O
        })

    return pd.DataFrame(harmonized_rows)


def harmonize_rfmid_v2(split: str) -> pd.DataFrame:
    base_path = "/kaggle/input/rdc-version-2/RFDiM2_0"

    if split == "train":
        img_dir = f"{base_path}/Training_set_2/Train_2"
        label_file = f"{base_path}/Training_set_2/RFMiD_2_Training_labels.csv"
    elif split == "val":
        img_dir = f"{base_path}/Validation_set_2/Validation_2"
        label_file = f"{base_path}/Validation_set_2/RFMiD_2_Validation_labels.csv"
    else:
        img_dir = f"{base_path}/Test_set_2/Test_2"
        label_file = f"{base_path}/Test_set_2/RFMiD_2_Testing_labels.csv"

    try:
        df = pd.read_csv(label_file, encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(label_file, encoding='latin1')

    df.columns = df.columns.str.strip()

    potential_disease_cols = ['AH', 'AION', 'ARMD', 'BRVO', 'CB', 'CF', 'CL', 'CME',
                              'CNV', 'CRAO', 'CRS', 'CRVO', 'CSR', 'CWS', 'CSC', 'DN',
                              'DR', 'EDN', 'ERM', 'GRT', 'HPED', 'HR', 'LS', 'MCA',
                              'ME', 'MH', 'MHL', 'MS', 'MYA', 'ODC', 'ODE', 'ODP',
                              'ON', 'OPDM', 'PRH', 'RD', 'RHL', 'RTR', 'RP', 'RPEC',
                              'RS', 'RT', 'SOFE', 'ST', 'TD', 'TSLN', 'TV', 'VS',
                              'HTN', 'IIH']

    all_disease_cols = [col for col in potential_disease_cols if col in df.columns]
    harmonized_rows = []

    skipped_count = 0
    found_count = 0

    for _, row in df.iterrows():
        image_id = int(row['ID'])

        image_path = None
        for ext in ['.jpg', '.JPG', '.png', '.PNG', '.jpeg', '.JPEG']:
            candidate = os.path.join(img_dir, f"{image_id}{ext}")
            if os.path.exists(candidate):
                image_path = candidate
                found_count += 1
                break

        if image_path is None:
            skipped_count += 1
            continue

        wnl = row.get('WNL', 0)
        N = 1 if wnl == 1 else 0
        D = 1 if row.get('DR', 0) == 1 else 0
        G = 1 if row.get('ODC', 0) == 1 else 0
        C = 1 if row.get('MH', 0) == 1 else 0
        A = 1 if row.get('ARMD', 0) == 1 else 0

        # HTN is your "H" mapping; fallback to HR if HTN absent
        H = 1 if row.get('HTN', 0) == 1 else 0
        if H == 0 and 'HR' in df.columns:
            H = 1 if row.get('HR', 0) == 1 else 0

        M = 1 if row.get('MYA', 0) == 1 else 0

        used_for_mapping = ['DR', 'ODC', 'MH', 'ARMD', 'HTN', 'HR', 'MYA', 'WNL']
        other_cols = [col for col in all_disease_cols if col not in used_for_mapping]
        O = 1 if any(row.get(col, 0) == 1 for col in other_cols) else 0

        harmonized_rows.append({
            'image_path': image_path,
            'dataset': 'RFMiD_v2',
            'split': split,
            'ID': image_id,
            'N': N, 'D': D, 'G': G, 'C': C, 'A': A, 'H': H, 'M': M, 'O': O
        })

    if skipped_count > 0:
        print(f"RFMiD_v2 {split}: Found {found_count}, Skipped {skipped_count}")

    return pd.DataFrame(harmonized_rows)


def harmonize_all_datasets() -> Dict[str, pd.DataFrame]:
    results = {}
    print("\n" + "=" * 60)
    print("HARMONIZING DATASETS")
    print("=" * 60)

    for split in ['train', 'val', 'test']:
        print(f"\nProcessing {split} split...")
        results[f'ODIR_{split}'] = harmonize_odir(split)
        results[f'RFMiD_v1_{split}'] = harmonize_rfmid_v1(split)
        results[f'RFMiD_v2_{split}'] = harmonize_rfmid_v2(split)

    return results


def save_harmonized_data(harmonized_data: Dict[str, pd.DataFrame], output_dir: str):
    os.makedirs(output_dir, exist_ok=True)

    print("\n" + "=" * 60)
    print("SAVING HARMONIZED DATA")
    print("=" * 60)

    for key, df in harmonized_data.items():
        output_path = os.path.join(output_dir, f"{key}.csv")
        df.to_csv(output_path, index=False)
        print(f"✅ Saved {key:20s}: {len(df):5d} rows → {output_path}")

    print("\n📦 Creating combined files...")
    for split in ['train', 'val', 'test']:
        split_dfs = [v for k, v in harmonized_data.items() if k.endswith(f'_{split}')]
        if split_dfs:
            combined = pd.concat(split_dfs, ignore_index=True)
            output_path = os.path.join(output_dir, f"combined_{split}.csv")
            combined.to_csv(output_path, index=False)
            print(f"✅ Saved combined_{split:5s}: {len(combined):5d} rows → {output_path}")

    print(f"\n✨ All files saved to: {output_dir}")


def verify_images(harmonized_data: Dict[str, pd.DataFrame]):
    print("\n" + "=" * 60)
    print("VERIFYING IMAGE PATHS")
    print("=" * 60)

    all_good = True
    for key, df in sorted(harmonized_data.items()):
        missing = df[~df['image_path'].apply(os.path.exists)]
        if len(missing) > 0:
            print(f"❌ {key:20s}: {len(missing)} missing images")
            all_good = False
        else:
            print(f"✅ {key:20s}: All {len(df)} images found")

    if all_good:
        print("\n🎉 All image paths verified successfully!")
    else:
        print("\n⚠️  Some images are missing (training will drop missing paths anyway).")


def ensure_harmonized_csvs(output_dir: str, required_csvs: list[str]):
    missing = [p for p in required_csvs if not os.path.exists(p)]
    if not missing:
        print(f"[INFO] Harmonized CSVs already exist in: {output_dir}")
        return

    print("\n[INFO] Missing harmonized CSVs. Generating them now...")
    for p in missing[:10]:
        print(f"  - missing: {p}")
    if len(missing) > 10:
        print(f"  ... (+{len(missing) - 10} more)")

    harmonized_data = harmonize_all_datasets()
    verify_images(harmonized_data)
    save_harmonized_data(harmonized_data, output_dir=output_dir)


# ============================================================
# 2) TRAINING PIPELINE (Mixup DG) + Swin-T
# ============================================================

# =============================================================================
# Config for FOLD (Swapped: Train=ODIR+RFMiD_v2, Test=RFMiD_v1)
# =============================================================================

SEED = 42
SAVE_DIR = '/kaggle/working/results_lodo_mixup/fold_test_RFMiD_v1_swin_t'
TEST_DOMAIN = "RFMiD_v1"
TRAIN_DOMAINS = "ODIR + RFMiD_v2"

MIXUP_ALPHA = 0.2
OUTPUT_LABEL_DIR = "/kaggle/working/harmonized_labels"

# Build CSV paths from OUTPUT_LABEL_DIR (so they match where we save)
TEST_CSV = os.path.join(OUTPUT_LABEL_DIR, "RFMiD_v1_test.csv")
TRAIN_CSVS = [
    os.path.join(OUTPUT_LABEL_DIR, "ODIR_train.csv"),
    os.path.join(OUTPUT_LABEL_DIR, "RFMiD_v2_train.csv"),
]
VAL_CSVS = [
    os.path.join(OUTPUT_LABEL_DIR, "ODIR_val.csv"),
    os.path.join(OUTPUT_LABEL_DIR, "RFMiD_v2_val.csv"),
]


# =============================================================================
# Reproducibility
# =============================================================================

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)


# =============================================================================
# Dataset (with domain tracking for Mixup!)
# =============================================================================

class RetinalDataset(Dataset):
    def __init__(self, csv_path, domain_id, transform=None):
        self.data = pd.read_csv(csv_path)
        self.domain_id = domain_id
        self.transform = transform
        self.label_cols = ["N", "D", "G", "C", "A", "H", "M", "O"]

        dup = self.data.duplicated(subset=["image_path"], keep="first").sum()
        if dup > 0:
            print(f"[WARN] {dup} duplicates found in {os.path.basename(csv_path)}, keeping first")
        self.data = self.data.drop_duplicates(subset=["image_path"]).reset_index(drop=True)

        valid_mask = self.data["image_path"].apply(os.path.exists)
        missing = (~valid_mask).sum()
        if missing > 0:
            print(f"[WARN] {os.path.basename(csv_path)}: dropping {missing} missing images")
        self.data = self.data.loc[valid_mask].reset_index(drop=True)

        print(f"[INFO] Domain {domain_id}: Loaded {len(self.data)} valid images from {os.path.basename(csv_path)}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        try:
            image = Image.open(row["image_path"]).convert("RGB")
        except:
            image = Image.new("RGB", (224, 224), color=(128, 128, 128))

        labels = torch.tensor(row[self.label_cols].values.astype("float32"))
        if self.transform:
            image = self.transform(image)
        return image, labels, self.domain_id


# =============================================================================
# Pos weight
# =============================================================================

def calculate_pos_weights(datasets, clip_min=0.5, clip_max=50.0):
    all_labels = []
    for ds in datasets:
        all_labels.append(ds.data[["N", "D", "G", "C", "A", "H", "M", "O"]].values)
    combined = np.vstack(all_labels)

    pos = combined.sum(axis=0)
    neg = len(combined) - pos
    raw = neg / (pos + 1e-5)
    clipped = np.clip(raw, clip_min, clip_max)

    print("\n[INFO] Positive class weights (from training domains):")
    for i, col in enumerate(["N", "D", "G", "C", "A", "H", "M", "O"]):
        print(f"  {col}: {clipped[i]:.2f}")

    return torch.tensor(clipped, dtype=torch.float32)


# =============================================================================
# Model (Swin Transformer Tiny) with version-safe loading
# =============================================================================

class SwinTMixupMultiLabel(nn.Module):
    def __init__(self, num_classes=8, dropout=0.3, pretrained=True):
        super().__init__()

        # Version-safe Swin-T loading (torchvision may differ across Kaggle images)
        if pretrained:
            try:
                weights = models.Swin_T_Weights.IMAGENET1K_V1
                self.backbone = models.swin_t(weights=weights)
            except Exception:
                self.backbone = models.swin_t(pretrained=True)
        else:
            try:
                self.backbone = models.swin_t(weights=None)
            except Exception:
                self.backbone = models.swin_t(pretrained=False)

        # Remove original head and replace
        in_features = self.backbone.head.in_features
        self.backbone.head = nn.Identity()

        self.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        feats = self.backbone(x)
        return self.classifier(feats)


# =============================================================================
# Transforms
# =============================================================================

def get_transforms(is_train=False):
    if is_train:
        return transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.RandomCrop(224),
            transforms.RandomRotation(10),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])
    else:
        return transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])


# =============================================================================
# Metrics
# =============================================================================

def compute_metrics(labels, probs, thresholds=None):
    n_classes = labels.shape[1]

    aucs = []
    for i in range(n_classes):
        if len(np.unique(labels[:, i])) > 1:
            aucs.append(roc_auc_score(labels[:, i], probs[:, i]))
        else:
            aucs.append(np.nan)

    aps = []
    for i in range(n_classes):
        if len(np.unique(labels[:, i])) > 1:
            aps.append(average_precision_score(labels[:, i], probs[:, i]))
        else:
            aps.append(np.nan)

    if thresholds is None:
        thresholds = np.full(n_classes, 0.5)

    preds = (probs >= thresholds).astype(int)
    f1 = f1_score(labels, preds, average="macro", zero_division=0)

    return {
        "mAUC": float(np.nanmean(aucs)),
        "mAP": float(np.nanmean(aps)),
        "per_class_auc": aucs,
        "per_class_ap": aps,
        "macro_f1": float(f1),
    }


def find_optimal_thresholds(labels, probs):
    n_classes = labels.shape[1]
    thresholds = []
    search_range = np.linspace(0.05, 0.95, 91)

    for i in range(n_classes):
        best_f1, best_t = 0.0, 0.5
        if len(np.unique(labels[:, i])) > 1:
            for t in search_range:
                preds = (probs[:, i] >= t).astype(int)
                f1 = f1_score(labels[:, i], preds, zero_division=0)
                if f1 > best_f1:
                    best_f1, best_t = f1, t
        thresholds.append(best_t)

    return np.array(thresholds)


# =============================================================================
# MIXUP TRAINING
# =============================================================================

def train_epoch_mixup(model, loaders_dict, criterion, optimizer, device, mixup_alpha=0.2):
    model.train()
    losses = []
    mixup_stats = {'total_batches': 0, 'mixup_batches': 0}

    domain_iters = {k: iter(v) for k, v in loaders_dict.items()}
    domain_ids = list(loaders_dict.keys())
    max_batches = max(len(loader) for loader in loaders_dict.values())

    pbar = tqdm(range(max_batches), desc="Train (Mixup)", leave=False)

    for _ in pbar:
        if len(domain_ids) >= 2 and np.random.rand() > 0.5:
            d1, d2 = np.random.choice(domain_ids, size=2, replace=False)

            try:
                imgs1, labels1, _ = next(domain_iters[d1])
            except StopIteration:
                domain_iters[d1] = iter(loaders_dict[d1])
                imgs1, labels1, _ = next(domain_iters[d1])

            try:
                imgs2, labels2, _ = next(domain_iters[d2])
            except StopIteration:
                domain_iters[d2] = iter(loaders_dict[d2])
                imgs2, labels2, _ = next(domain_iters[d2])

            min_size = min(imgs1.size(0), imgs2.size(0))
            imgs1, labels1 = imgs1[:min_size], labels1[:min_size]
            imgs2, labels2 = imgs2[:min_size], labels2[:min_size]

            lam = np.random.beta(mixup_alpha, mixup_alpha)

            mixed_imgs = lam * imgs1 + (1 - lam) * imgs2
            mixed_labels = lam * labels1 + (1 - lam) * labels2

            mixed_imgs = mixed_imgs.to(device)
            mixed_labels = mixed_labels.to(device)

            optimizer.zero_grad()
            logits = model(mixed_imgs)
            loss = criterion(logits, mixed_labels)

            mixup_stats['mixup_batches'] += 1
        else:
            d = np.random.choice(domain_ids)
            try:
                imgs, labels, _ = next(domain_iters[d])
            except StopIteration:
                domain_iters[d] = iter(loaders_dict[d])
                imgs, labels, _ = next(domain_iters[d])

            imgs, labels = imgs.to(device), labels.to(device)

            optimizer.zero_grad()
            logits = model(imgs)
            loss = criterion(logits, labels)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        losses.append(loss.item())
        mixup_stats['total_batches'] += 1
        pbar.set_postfix({'loss': f"{loss.item():.4f}"})

    return {
        "loss": float(np.mean(losses)),
        "mixup_ratio": mixup_stats['mixup_batches'] / max(1, mixup_stats['total_batches'])
    }


@torch.no_grad()
def validate(model, loader, criterion, device, thresholds=None):
    model.eval()
    losses, all_probs, all_labels = [], [], []

    for batch in tqdm(loader, desc="Val", leave=False):
        if len(batch) == 3:
            images, labels, _ = batch
        else:
            images, labels = batch

        images, labels = images.to(device), labels.to(device)
        logits = model(images)
        loss = criterion(logits, labels)

        losses.append(loss.item())
        all_probs.append(torch.sigmoid(logits).cpu().numpy())
        all_labels.append(labels.cpu().numpy())

    probs = np.vstack(all_probs)
    labels = np.vstack(all_labels)
    metrics = compute_metrics(labels, probs, thresholds)
    metrics["loss"] = float(np.mean(losses))
    return metrics, probs, labels


# =============================================================================
# Visualizations
# =============================================================================

def plot_training_curves(history, save_dir):
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    epochs = range(1, len(history['train_loss']) + 1)

    axes[0].plot(epochs, history['train_loss'], marker='o', linewidth=2, label='Train Loss')
    axes[0].plot(epochs, history['val_loss'], marker='s', linewidth=2, label='Val Loss')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Training & Validation Loss (Mixup, Swin-T)')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    axes[1].plot(epochs, history['val_auc'], marker='s', linewidth=2, label='Val mAUC')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('mAUC')
    axes[1].set_title('Validation mAUC (Mixup, Swin-T)')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    axes[1].set_ylim([0, 1.0])

    os.makedirs(save_dir, exist_ok=True)
    plt.tight_layout()
    plt.savefig(f'{save_dir}/training_curves.png', dpi=300, bbox_inches='tight')
    plt.close()
    print(f"[INFO] ✓ Saved training curves")


def plot_per_class_roc(labels, probs, test_domain, save_dir):
    class_names = ["N", "D", "G", "C", "A", "H", "M", "O"]
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    axes = axes.flatten()

    for i in range(8):
        if len(np.unique(labels[:, i])) > 1:
            fpr, tpr, _ = roc_curve(labels[:, i], probs[:, i])
            auc = roc_auc_score(labels[:, i], probs[:, i])
            axes[i].plot(fpr, tpr, linewidth=2, label=f'AUC = {auc:.3f}')
            axes[i].plot([0, 1], [0, 1], 'k--', linewidth=1, alpha=0.5)
            axes[i].set_title(f'Class {class_names[i]}')
            axes[i].set_xlabel('FPR')
            axes[i].set_ylabel('TPR')
            axes[i].legend(loc='lower right')
            axes[i].grid(True, alpha=0.3)
        else:
            axes[i].text(0.5, 0.5, 'Single class\n(No ROC)', ha='center', va='center')
            axes[i].set_title(f'Class {class_names[i]}')

    plt.suptitle(f'Per-Class ROC Curves - Test on {test_domain} (Mixup, Swin-T)', y=0.995)
    os.makedirs(save_dir, exist_ok=True)
    plt.tight_layout()
    plt.savefig(f'{save_dir}/per_class_roc_curves.png', dpi=300, bbox_inches='tight')
    plt.close()
    print(f"[INFO] ✓ Saved per-class ROC curves")


def plot_macro_roc(labels, probs, test_domain, save_dir):
    all_fpr, all_tpr = [], []
    for i in range(8):
        if len(np.unique(labels[:, i])) > 1:
            fpr, tpr, _ = roc_curve(labels[:, i], probs[:, i])
            all_fpr.append(fpr)
            all_tpr.append(tpr)

    mean_fpr = np.linspace(0, 1, 100)
    tprs = [np.interp(mean_fpr, fpr, tpr) for fpr, tpr in zip(all_fpr, all_tpr)]
    mean_tpr = np.mean(tprs, axis=0)
    macro_auc = np.trapz(mean_tpr, mean_fpr)

    plt.figure(figsize=(8, 6))
    plt.plot(mean_fpr, mean_tpr, linewidth=3, label=f'Macro ROC (AUC = {macro_auc:.3f})')
    plt.plot([0, 1], [0, 1], 'k--', linewidth=2, alpha=0.5, label='Random')
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.title(f'Macro ROC - Test on {test_domain} (Mixup, Swin-T)')
    plt.legend(loc='lower right')
    plt.grid(True, alpha=0.3)

    os.makedirs(save_dir, exist_ok=True)
    plt.tight_layout()
    plt.savefig(f'{save_dir}/macro_roc_curve.png', dpi=300, bbox_inches='tight')
    plt.close()
    print(f"[INFO] ✓ Saved macro ROC curve")


def plot_per_class_metrics(test_metrics, save_dir):
    class_names = ["N", "D", "G", "C", "A", "H", "M", "O"]
    aucs = [0 if np.isnan(x) else x for x in test_metrics['per_class_auc']]
    aps = [0 if np.isnan(x) else x for x in test_metrics['per_class_ap']]

    x = np.arange(len(class_names))
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    axes[0].bar(x, aucs, alpha=0.8, edgecolor='black', linewidth=1.2)
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(class_names)
    axes[0].set_ylim([0, 1.0])
    axes[0].set_title('Per-Class AUC (Mixup, Swin-T)')
    axes[0].grid(True, alpha=0.3, axis='y')

    axes[1].bar(x, aps, alpha=0.8, edgecolor='black', linewidth=1.2)
    axes[1].set_xticks(x)
    axes[1].set_xticklabels(class_names)
    axes[1].set_ylim([0, 1.0])
    axes[1].set_title('Per-Class AP (Mixup, Swin-T)')
    axes[1].grid(True, alpha=0.3, axis='y')

    os.makedirs(save_dir, exist_ok=True)
    plt.tight_layout()
    plt.savefig(f'{save_dir}/per_class_metrics.png', dpi=300, bbox_inches='tight')
    plt.close()
    print(f"[INFO] ✓ Saved per-class metrics")


# =============================================================================
# MAIN
# =============================================================================

if __name__ == "__main__":
    set_seed(SEED)
    os.makedirs(SAVE_DIR, exist_ok=True)

    # FIX: ensure the required harmonized CSVs exist BEFORE reading them
    required = TRAIN_CSVS + VAL_CSVS + [TEST_CSV]
    ensure_harmonized_csvs(output_dir=OUTPUT_LABEL_DIR, required_csvs=required)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    print("=" * 80)
    print(f"LODO FOLD (SWAPPED) with MIXUP DG: Test on {TEST_DOMAIN}")
    print(f"Training on: {TRAIN_DOMAINS}")
    print("=" * 80)
    print(f"Device: {device}")
    print(f"Seed: {SEED}")
    print(f"Mixup Alpha: {MIXUP_ALPHA}")
    print("=" * 80)

    print("\n[INFO] Loading datasets...")
    train_datasets = [RetinalDataset(csv, domain_id=i, transform=get_transforms(True))
                      for i, csv in enumerate(TRAIN_CSVS)]
    val_datasets = [RetinalDataset(csv, domain_id=i, transform=get_transforms(False))
                    for i, csv in enumerate(VAL_CSVS)]
    test_dataset = RetinalDataset(TEST_CSV, domain_id=999, transform=get_transforms(False))

    print(f"\n[INFO] Total train: {sum(len(ds) for ds in train_datasets)} images")
    print(f"[INFO] Total val: {sum(len(ds) for ds in val_datasets)} images")
    print(f"[INFO] Test: {len(test_dataset)} images")

    pos_weights = calculate_pos_weights(train_datasets).to(device)

    # DataLoaders (separate per domain for mixup)
    g = torch.Generator().manual_seed(SEED)
    train_loaders = {
        i: DataLoader(ds, batch_size=32, shuffle=True, num_workers=4,
                      pin_memory=True, generator=g)
        for i, ds in enumerate(train_datasets)
    }

    combined_val = ConcatDataset(val_datasets)
    val_loader = DataLoader(combined_val, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

    print("\n[INFO] Initializing Swin-T model...")
    model = SwinTMixupMultiLabel(num_classes=8, dropout=0.3, pretrained=True).to(device)

    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weights)

    # AdamW is common for transformers
    optimizer = optim.AdamW([
        {'params': model.backbone.parameters(), 'lr': 5e-5},
        {'params': model.classifier.parameters(), 'lr': 5e-4}
    ], weight_decay=1e-4)

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)

    history = {'train_loss': [], 'val_loss': [], 'val_auc': []}
    best_val_auc, patience_counter = 0.0, 0

    print("\n[INFO] Training started with Mixup DG (Swin-T)...")
    print("-" * 80)

    for epoch in range(50):
        train_metrics = train_epoch_mixup(model, train_loaders, criterion, optimizer, device, mixup_alpha=MIXUP_ALPHA)
        val_metrics, _, _ = validate(model, val_loader, criterion, device)
        scheduler.step(val_metrics['mAUC'])

        history['train_loss'].append(train_metrics['loss'])
        history['val_loss'].append(val_metrics['loss'])
        history['val_auc'].append(val_metrics['mAUC'])

        print(f"Epoch {epoch+1:02d} | Train Loss: {train_metrics['loss']:.4f} "
              f"| Val mAUC: {val_metrics['mAUC']:.4f} | Mixup: {train_metrics['mixup_ratio']:.1%}")

        if val_metrics['mAUC'] > best_val_auc:
            best_val_auc = val_metrics['mAUC']
            torch.save(model.state_dict(), f'{SAVE_DIR}/best_model.pth')
            print(f"  ✓ Saved best model (val mAUC: {best_val_auc:.4f})")
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= 10:
            print(f"\n[INFO] Early stopping at epoch {epoch+1}")
            break

    print(f"\n[INFO] Loading best model...")
    model.load_state_dict(torch.load(f'{SAVE_DIR}/best_model.pth', map_location=device))
    print(f"[INFO] Best validation mAUC: {best_val_auc:.4f}")

    print(f"\n[INFO] Finding optimal thresholds on validation...")
    _, val_probs, val_labels = validate(model, val_loader, criterion, device)
    thresholds = find_optimal_thresholds(val_labels, val_probs)

    class_names = ["N", "D", "G", "C", "A", "H", "M", "O"]
    print("[INFO] Optimal thresholds:")
    for c, t in zip(class_names, thresholds):
        print(f"  {c}: {t:.3f}")

    print(f"\n[INFO] Testing on {TEST_DOMAIN}...")
    print("-" * 80)
    test_metrics, test_probs, test_labels = validate(model, test_loader, criterion, device, thresholds)

    print("\n" + "=" * 80)
    print(f"TEST RESULTS - {TEST_DOMAIN} (Mixup DG, Swin-T)")
    print("=" * 80)
    print(f"mAUC:      {test_metrics['mAUC']:.4f}")
    print(f"mAP:       {test_metrics['mAP']:.4f}")
    print(f"Macro F1:  {test_metrics['macro_f1']:.4f}")
    print("=" * 80)

    print(f"\n{'Class':<8} {'AUC':<10} {'AP':<10}")
    print("-" * 30)
    for i, cls in enumerate(class_names):
        auc = test_metrics['per_class_auc'][i]
        ap = test_metrics['per_class_ap'][i]
        auc_str = f"{auc:.4f}" if not np.isnan(auc) else "N/A"
        ap_str = f"{ap:.4f}" if not np.isnan(ap) else "N/A"
        print(f"{cls:<8} {auc_str:<10} {ap_str:<10}")
    print("-" * 30)

    print("\n[INFO] Saving results...")
    results_df = pd.DataFrame([{
        'method': 'Mixup',
        'backbone': 'swin_t',
        'test_domain': TEST_DOMAIN,
        'train_domains': TRAIN_DOMAINS,
        'mAUC': test_metrics['mAUC'],
        'mAP': test_metrics['mAP'],
        'macro_f1': test_metrics['macro_f1'],
        'best_val_auc': best_val_auc,
        'mixup_alpha': MIXUP_ALPHA
    }])
    results_df.to_csv(f'{SAVE_DIR}/test_results.csv', index=False)
    print(f"[INFO] ✓ Saved test_results.csv")

    print("\n[INFO] Generating visualizations...")
    plot_training_curves(history, SAVE_DIR)
    plot_per_class_roc(test_labels, test_probs, TEST_DOMAIN, SAVE_DIR)
    plot_macro_roc(test_labels, test_probs, TEST_DOMAIN, SAVE_DIR)
    plot_per_class_metrics(test_metrics, SAVE_DIR)

    print("\n" + "=" * 80)
    print("✓ MIXUP LODO FOLD (SWAPPED) COMPLETE!")
    print(f"✓ Results saved to: {SAVE_DIR}/")
    print("=" * 80)



[INFO] Missing harmonized CSVs. Generating them now...
  - missing: /kaggle/working/harmonized_labels/ODIR_train.csv
  - missing: /kaggle/working/harmonized_labels/RFMiD_v2_train.csv
  - missing: /kaggle/working/harmonized_labels/ODIR_val.csv
  - missing: /kaggle/working/harmonized_labels/RFMiD_v2_val.csv
  - missing: /kaggle/working/harmonized_labels/RFMiD_v1_test.csv

HARMONIZING DATASETS

Processing train split...
RFMiD_v2 train: Found 507, Skipped 2

Processing val split...

Processing test split...
RFMiD_v2 test: Found 170, Skipped 4

VERIFYING IMAGE PATHS
✅ ODIR_test           : All 2000 images found
✅ ODIR_train          : All 7000 images found
✅ ODIR_val            : All 1000 images found
✅ RFMiD_v1_test       : All 640 images found
✅ RFMiD_v1_train      : All 1920 images found
✅ RFMiD_v1_val        : All 640 images found
✅ RFMiD_v2_test       : All 170 images found
✅ RFMiD_v2_train      : All 507 images found
✅ RFMiD_v2_val        : All 177 images found

🎉 All image paths ver

100%|██████████| 108M/108M [00:00<00:00, 164MB/s]  



[INFO] Training started with Mixup DG (Swin-T)...
--------------------------------------------------------------------------------


Epoch 01 | Train Loss: 0.9625 | Val mAUC: 0.7745 | Mixup: 53.9%
  ✓ Saved best model (val mAUC: 0.7745)


Epoch 02 | Train Loss: 0.7650 | Val mAUC: 0.8071 | Mixup: 44.3%
  ✓ Saved best model (val mAUC: 0.8071)


...

Epoch 26 | Train Loss: 0.4310 | Val mAUC: 0.8294 | Mixup: 51.6%

[INFO] Early stopping at epoch 26

[INFO] Loading best model...
[INFO] Best validation mAUC: 0.8315

[INFO] Finding optimal thresholds on validation...


[INFO] Optimal thresholds:
  N: 0.290
  D: 0.210
  G: 0.750
  C: 0.950
  A: 0.910
  H: 0.950
  M: 0.950
  O: 0.530

[INFO] Testing on RFMiD_v1...
--------------------------------------------------------------------------------



TEST RESULTS - RFMiD_v1 (Mixup DG, Swin-T)
mAUC:      0.8690
mAP:       0.5245
Macro F1:  0.4176

Class    AUC        AP        
------------------------------
N        0.9370     0.7462    
D        0.9060     0.6821    
G        0.7342     0.4316    
C        0.9013     0.7460    
A        0.9030     0.4377    
H        0.9531     0.0323    
M        0.9492     0.5532    
O        0.6682     0.5671    
------------------------------

[INFO] Saving results...
[INFO] ✓ Saved test_results.csv

[INFO] Generating visualizations...
[INFO] ✓ Saved training curves
[INFO] ✓ Saved per-class ROC curves


/tmp/ipykernel_55/3624701511.py:690: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  macro_auc = np.trapz(mean_tpr, mean_fpr)


[INFO] ✓ Saved macro ROC curve
[INFO] ✓ Saved per-class metrics

✓ MIXUP LODO FOLD (SWAPPED) COMPLETE!
✓ Results saved to: /kaggle/working/results_lodo_mixup/fold_test_RFMiD_v1_swin_t/
